# Get Historical Market Cap Data

https://site.financialmodelingprep.com/developer/docs#historical-market-cap-company-information

format: https://financialmodelingprep.com/api/v3/historical-market-capitalization/AAPL?limit=100&from=2023-10-10&to=2023-12-10

each query is 5 years only

In [10]:
import pandas as pd
import requests

# Load the CSV file into a pandas DataFrame
df = pd.read_csv("~/Box/STAT 222 Capstone/Intermediate Data/Credit_Rating/combined_credit_rating_data.csv")

# Extract all distinct "Symbol" values
distinct_symbols = df["Symbol"].unique()

# Convert the array of distinct symbols to a pandas list
distinct_symbols_list = distinct_symbols.tolist()

print(len(distinct_symbols_list))

886


In [11]:
# Create symbol start and end dates for individual requests
group_1_dates = ["2010-01-01", "2013-12-31"]
group_2_dates = ["2014-01-01", "2016-12-31"]

# Dataframe of distinct symbols_list and group_1_dates as two columns
df1 = pd.DataFrame(distinct_symbols_list, columns=["Symbol"])
df1["Start_Date"] = group_1_dates[0]
df1["End_Date"] = group_1_dates[1]

# Dataframe of distinct symbols_list and group_2_dates as two columns
df2 = pd.DataFrame(distinct_symbols_list, columns=["Symbol"])
df2["Start_Date"] = group_2_dates[0]
df2["End_Date"] = group_2_dates[1]

# Concatenate the two dataframes
api_requests_df = pd.concat([df1, df2]).sort_values(by=["Symbol", "Start_Date"]).reset_index(drop=True)
api_requests_df


,Symbol,Start_Date,End_Date
0,AA,2010-01-01,2013-12-31
1,AA,2014-01-01,2016-12-31
2,AAL,2010-01-01,2013-12-31
3,AAL,2014-01-01,2016-12-31
4,AAP,2010-01-01,2013-12-31
...,...,...,...
1767,YUM,2014-01-01,2016-12-31
1768,ZBRA,2010-01-01,2013-12-31
1769,ZBRA,2014-01-01,2016-12-31
1770,ZTS,2010-01-01,2013-12-31


In [12]:
# API key for accessing the financial data
api_key = "GzzXB8zIm8u1NJzcFKZNOwcOyxarZEmi"

# Function to fetch market cap data during period for company
def fetch_market_cap(company, start_date, end_date):
    market_cap = requests.get(f'https://financialmodelingprep.com/api/v3/historical-market-capitalization/{company}?from={start_date}&to={end_date}&apikey={api_key}')
    market_cap = market_cap.json()
    return market_cap

# tqdm for progress bar
#from tqdm.notebook import tqdm

# Iterate over dataframe rows
# May need to break this up into smaller chunks to avoid API rate limits
market_cap_data = [] # Initialize empty list to store the fetched data
for index, row in api_requests_df.iterrows():

    # Fetch market cap data
    company = row["Symbol"]
    start_date = row["Start_Date"]
    end_date = row["End_Date"]
    market_cap = fetch_market_cap(company, start_date, end_date)
    
    # Append fetched data to list
    market_cap_data.extend(market_cap)

# Combine data into a single DataFrame
market_cap_data = pd.DataFrame(market_cap_data)
market_cap_data

KeyboardInterrupt: 

In [ ]:
# Save parquet file
market_cap_data.to_parquet("~\Box\STAT 222 Capstone\Raw Data\Market Cap Data\market_cap_data.parquet")